<a href="https://colab.research.google.com/github/GummyBear-w/aop113b/blob/main/final_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 專案報告：寶可夢問答遊戲



## 一、專案動機與目的



本專案以我個人喜歡的寶可夢為主題，設計並實作一款支援多人連線的網頁寶可夢問答遊戲。

遊戲的核心機制建立於 PokéAPI 公開資料之上，透過串接 API 即時取得寶可夢圖片與相關資訊作為題目素材，實現資料驅動的互動體驗。本專案整合 React 前端框架、Socket.IO 即時通訊技術，以及 Render 雲端平台部署，實踐完整的前後端整合流程與多人互動遊戲體驗。



## 二、系統架構與開發工具



### 前端技術
- React（Vite 建構工具）
- Tailwind CSS（UI 樣式設計）
- Socket.IO client（即時通訊）
- Vercel（前端部署平台）

### 後端技術
- Node.js + Express + Socket.IO（即時伺服器）
- Render（免費雲端後端部署）




## 三、功能介紹



### 遊戲模式
- 單人模式：玩家根據圖片輸入寶可夢名稱，答題完畢後顯示得分。
- 多人模式：支援多人連線遊玩（至多5人），和朋友一起搶答，最後以排行榜顯示誰才是真正的贏家。

### 自訂設定
- 題數與答題秒數可透過滑桿設定。
- 支援明亮與暗色主題切換。

### 答案氣泡
- 玩家答題會以浮動氣泡顯示於畫面，提供干擾效果增加遊戲難度同時顯示先前回答。
- 答對為黃色氣泡，答錯為橘色。

### 房間機制
- 玩家可創建或加入房間進行多人遊戲。
- 由房主設定題數與秒數，並開始遊戲。




## 四、成果展示


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 主要程式片段

**呼叫 PokéAPI 獲取寶可夢**  
從 PokeAPI 取得隨機寶可夢資料，解析圖片與多語系名稱，並回傳作為問答資料使用。

In [ ]:
async function fetchRandomPokemon() {
	try {
		const id = Math.floor(Math.random() * 1010) + 1;
		console.log(`🔍 獲取寶可夢 ID: ${id}`);

		const res = await fetch(`https://pokeapi.co/api/v2/pokemon/${id}`);
		const data = await res.json();

		const speciesRes = await fetch(data.species.url);
		const speciesData = await speciesRes.json();

		const zhEntry = speciesData.names.find(
			(n) => n.language.name === "zh-Hant"
		);
		const nameZh = zhEntry?.name || speciesData.names[0]?.name || "未知";
		const allNames = speciesData.names.map((n) => n.name);
		allNames.push(data.name);

		console.log(`✅ 已獲取寶可夢: ${nameZh}`);

		return {
			imageUrl: data.sprites.other["official-artwork"].front_default,
			correctAnswer: nameZh,
			acceptedAnswers: allNames.map((n) => n.toLowerCase()),
		};
	} catch (error) {
		console.error(`❌ 獲取寶可夢失敗:`, error);
		throw error;
	}
}

**答案提交與即時回饋**  
處理玩家即時提交答案、判斷是否正確、更新分數、並廣播正確答案與觸發下一題。

In [ ]:
	socket.on("submit_answer", async ({ answer }) => {
		const roomCode = socketToRoom[socket.id];
		const room = rooms[roomCode];
		if (!room || !room.gameInProgress) return;

		const player = room.players.find((p) => p.id === socket.id);
		if (!player) return;

		if (room.questionAnswered) {
			return;
		}

		const userAnswer = answer.toLowerCase().trim();

		if (!userAnswer) {
			io.to(roomCode).emit("player_answered", {
				playerId: socket.id,
				nickname: player.nickname,
				answer,
				correct: false,
			});
			return;
		}

		const acceptedAnswers = room.currentQuestionData?.acceptedAnswers || [];

		let isCorrect = false;
		for (const acceptedAnswer of acceptedAnswers) {
			if (
				userAnswer === acceptedAnswer ||
				userAnswer.includes(acceptedAnswer) ||
				acceptedAnswer.includes(userAnswer)
			) {
				isCorrect = true;
				break;
			}
		}

		io.to(roomCode).emit("player_answered", {
			playerId: socket.id,
			nickname: player.nickname,
			answer,
			correct: isCorrect,
		});

		if (isCorrect) {

			room.questionAnswered = true;

			player.score += 1;

			io.to(roomCode).emit("game_update", {
				players: room.players,
			});

			clearAllTimers(room);

			io.to(roomCode).emit("show_answer", {
				correctAnswer: room.currentQuestionData.correctAnswer,
				answeredBy: player.nickname,
			});

			room.nextQuestionTimer = setTimeout(() => {
				startNextQuestion(roomCode);
			}, 1500);
		}
	});

完整程式碼在Github：[前端](https://github.com/GummyBear-w/poke-quiz-game)、[後端](https://github.com/GummyBear-w/poke-quiz-server)。

### 單人遊戲demo影片

<video src="https://github.com/GummyBear-w/poke-quiz-game/raw/main/img/%E5%96%AE%E4%BA%BA%E9%81%8A%E6%88%B2%E7%95%AB%E9%9D%A2.mp4" controls width="750"></video>

### 多人遊戲demo影片

<video src="https://github.com/GummyBear-w/poke-quiz-game/raw/main/img/%E5%A4%9A%E4%BA%BA%E9%81%8A%E6%88%B2%E7%95%AB%E9%9D%A2.mp4" controls width="750"></video>

### 詳細遊戲介面展示

**選擇遊玩模式**
<br>
<img src="https://github.com/GummyBear-w/poke-quiz-game/raw/main/img/%E9%81%B8%E6%93%87%E6%A8%A1%E5%BC%8F.png" width= "750px" >

**輸入暱稱**
<br>
<img src="https://github.com/GummyBear-w/poke-quiz-game/raw/main/img/%E8%BC%B8%E5%85%A5%E6%9A%B1%E7%A8%B1.png" width= "750px" >

**選擇創建或加入房間**
<br>
<img src="https://github.com/GummyBear-w/poke-quiz-game/raw
/main/img/%E9%81%B8%E6%93%87%E7%95%B6%E6%88%BF%E4%B8%BB%E9%82%84%E6%98%AF%E5%8A%A0%E5%85%A5.png" width= "750px" >

**房主介面**
<br>
<img src="https://github.com/GummyBear-w/poke-quiz-game/raw/main/img/%E6%88%BF%E4%B8%BB%E7%95%8C%E9%9D%A2.png" width= "750px" >

**加入者介面（dark mode）**
<br>
<img src="https://github.com/GummyBear-w/poke-quiz-game/raw/main/img/%E5%B7%B2%E5%8A%A0%E5%85%A5.png" width= "750px" >

**由房主開始遊戲**
<br>
<img src="https://github.com/GummyBear-w/poke-quiz-game/raw/main/img/%E7%94%B1%E6%88%BF%E4%B8%BB%E9%96%8B%E4%BD%BF%E9%81%8A%E6%88%B2.png" width= "750px" >

**多人遊戲進行畫面**
<br>
<img src="https://github.com/GummyBear-w/poke-quiz-game/raw/main/img/%E6%88%AA%E5%9C%96%202025-06-15%20%E6%99%9A%E4%B8%8A8.45.17.png" width= "750px" >

**多人遊戲結束，排行榜畫面**
<br>
<img src="https://github.com/GummyBear-w/poke-quiz-game/raw/main/img/%E6%88%AA%E5%9C%96%202025-06-15%20%E6%99%9A%E4%B8%8A8.45.40.png" width= "750px" >


## 五、限制與未來展望



### 限制
- Render 免費伺服器一定時間後將休眠導致初次連線時間等待時間偏長，免費額度耗盡後伺服器會停止。
- 實作連線狀態偵測、自動重連與手動跳過機制，確保遊戲不中斷。
- 目前尚未支援玩家中途斷線後重新加入同一場遊戲。
- 題目資料完全來自第三方 API，若 PokéAPI 停止提供服務，遊戲將無法正常產生題目。

### 未來展望
- 增加自定義功能，如開關答題提示、設定等待時間、踢除違規玩家、延遲開始遊戲等，提升管理彈性。
- 最佳化手機與平板操作介面，加入虛擬鍵盤按鈕、語音輸入等更適合行動裝置的答題模式。
- 建設資料庫讓玩家能保留自己的遊玩紀錄

## 也許有人想玩


想玩的話請到 ➡️ [寶可夢問答遊戲](
https://poke-quiz-game-pied.vercel.app/) 由於我是免費仔所以多人遊戲剛開始建房間可能要等半分鐘左右，請見諒。    
單人模式（寶可夢問答大賽）就不用等哦！
